In [1]:
import pickle
import pandas as pd
import re

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report 
from sklearn.decomposition import NMF,LatentDirichletAllocation
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [3]:
us = pickle.load(open("wikivoyage_text_US.p","rb"))

In [4]:
df = pd.DataFrame.from_dict(us,orient='index')

In [5]:
df = df.reset_index()

In [6]:
df = df.rename(columns=({ 'index' : 'Name'}))

In [7]:
df.head()

,Name,text,type,loc
0,AbercrombiePAGE,{{pagebanner|Abercrombie WikiVoyage Banner ND....,page,//tools.wmflabs.org/wikivoyage/w/poimap2.php?l...
1,Aberdeen (Maryland)PAGE,{{pagebanner|Aberdeen MD WikiVoyage Banner.jpg...,page,//tools.wmflabs.org/wikivoyage/w/poimap2.php?l...
2,Aberdeen (South Dakota)PAGE,{{pagebanner|Pagebanner default.jpg|pgname=Abe...,page,//tools.wmflabs.org/wikivoyage/w/poimap2.php?l...
3,Aberdeen (Washington)PAGE,{{Pagebanner|pgname=Aberdeen |Wikivoyage page ...,page,//tools.wmflabs.org/wikivoyage/w/poimap2.php?l...
4,AbernathyPAGE,{{pagebanner|Abernathy Texas Wikivoyage Banner...,page,//tools.wmflabs.org/wikivoyage/w/poimap2.php?l...


In [8]:
def unwiki(wiki):
        """
       Remove wiki markup from the text.
       """
        wiki = re.sub(r'(?i)\{\{IPA(\-[^\|\{\}]+)*?\|([^\|\{\}]+)(\|[^\{\}]+)*?\}\}', lambda m: m.group(2), wiki)
        wiki = re.sub(r'(?i)\{\{Lang(\-[^\|\{\}]+)*?\|([^\|\{\}]+)(\|[^\{\}]+)*?\}\}', lambda m: m.group(2), wiki)
        wiki = re.sub(r'\{\{[^\{\}]+\}\}', '', wiki)
        wiki = re.sub(r'(?m)\{\{[^\{\}]+\}\}', '', wiki)
        wiki = re.sub(r'(?m)\{\|[^\{\}]*?\|\}', '', wiki)
        wiki = re.sub(r'(?i)\[\[Category:[^\[\]]*?\]\]', '', wiki)
        wiki = re.sub(r'(?i)\[\[Image:[^\[\]]*?\]\]', '', wiki)
        wiki = re.sub(r'(?i)\[\[File:[^\[\]]*?\]\]', '', wiki)
        wiki = re.sub(r'\[\[[^\[\]]*?\|([^\[\]]*?)\]\]', lambda m: m.group(1), wiki)
        wiki = re.sub(r'\[\[([^\[\]]+?)\]\]', lambda m: m.group(1), wiki)
        wiki = re.sub(r'\[\[([^\[\]]+?)\]\]', '', wiki)
        wiki = re.sub(r'(?i)File:[^\[\]]*?', '', wiki)
        wiki = re.sub(r'\[[^\[\]]*? ([^\[\]]*?)\]', lambda m: m.group(1), wiki)
        wiki = re.sub(r"''+", '', wiki)
        wiki = re.sub(r'(?m)^\*$', '', wiki)
       
        return wiki

In [9]:
def unhtml(html):
        """
       Remove HTML from the text.
       """
        html = re.sub(r'(?i)&nbsp;', ' ', html)
        html = re.sub(r'(?i)<br[ \\]*?>', '\n', html)
        html = re.sub(r'(?m)<!--.*?--\s*>', '', html)
        html = re.sub(r'(?i)<ref[^>]*>[^>]*<\/ ?ref>', '', html)
        html = re.sub(r'(?m)<.*?>', '', html)
        html = re.sub(r'(?i)&amp;', '&', html)
       
        return html

In [10]:
def f(x):
    return unhtml(unwiki(x))

In [11]:
df['text'] = df['text'].apply(f)

In [ ]:
vect = CountVectorizer(stop_words="english", max_features = 5000)

In [ ]:
features = df['text']

In [ ]:
x_features = vect.fit_transform(features)

In [ ]:
nmf = NMF(n_components=20, random_state=1, alpha=0, l1_ratio=0)
W = nmf.fit_transform(x_features)
H = nmf.components_

In [ ]:
vector_features = vect.get_feature_names()

In [36]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print "Topic #%d:" % topic_idx
        print " ".join(sorted([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]],key=lambda x: len(x),reverse=True))

In [ ]:
print_top_words(nmf, vector_features, 50)

In [ ]:
for topic in range(W.shape[1]):
    #max_t = np.argmax(W[:,topic])
    print "TOPIC: #",topic
    #print features[max_t][0:150]
    
    for t in range(10):
        n = int(-1-t)
        max_2t = W[:,topic].argsort()[n]
        print df['Name'][max_2t]

In [ ]:
list(df[df['Name']=='Touring prestigious and notable universities in the U.S.PAGE']['loc'])

In [12]:
pages = df[df['type'] == 'page']

In [13]:
def f(x):
    return re.sub('PAGE', '', x)

In [14]:
pages['Name'] = pages['Name'].apply(f)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [ ]:
len("//tools.wmflabs.org/wikivoyage/w/poimap2")

In [ ]:
pages.head()

In [15]:
pages = pages[pages['loc'].str.contains("//tools.wmflabs.org/wikivoyage/w/poimap2")==True]

In [16]:
from nltk.corpus import stopwords

In [17]:
StopWords = stopwords.words('english')

In [18]:
StopPlaces = list(pages['Name'])

In [19]:
StopPlaces = map(lambda x: re.sub("\(|\)", '',x), StopPlaces)

In [20]:
StopPlaces = map(lambda x: x.lower().split(), StopPlaces)

In [21]:
StopPlaces = [item for sublist in StopPlaces for item in sublist]

In [22]:
StopWords = StopPlaces + StopWords

In [23]:
StopWords = list(set(StopWords))

In [ ]:
vect = CountVectorizer(stop_words=StopWords)

In [ ]:
features = pages['text'].apply(lambda x: x.lower())

In [ ]:
x_features = vect.fit_transform(features)

In [ ]:
nmf = NMF(n_components=20, random_state=1, alpha=0, l1_ratio=0)
W = nmf.fit_transform(x_features)
H = nmf.components_

In [ ]:
vector_features = vect.get_feature_names()

In [ ]:
print_top_words(nmf, vector_features, 50)

In [ ]:
for topic in range(W.shape[1]):
    #max_t = np.argmax(W[:,topic])
    print "TOPIC: #",topic
    #print features[max_t][0:150]
    
    for t in range(10):
        n = int(-1-t)
        max_2t = W[:,topic].argsort()[n]
        print list(pages['Name'])[max_2t]

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

In [ ]:
### TF-IDF Vectorizer

In [25]:
StopWords = StopWords + map(lambda x: x.split("/"), StopWords)

In [26]:
StopWords = [item for sublist in StopWords for item in sublist]

In [ ]:
TFIDFvect = TfidfVectorizer(stop_words=StopWords)

In [ ]:
x_features = TFIDFvect.fit_transform(features)

In [ ]:
nmf = NMF(n_components=20, random_state=1, alpha=0, l1_ratio=0)
W = nmf.fit_transform(x_features)
H = nmf.components_

In [ ]:
vector_features = TFIDFvect.get_feature_names()

In [ ]:
print_top_words(nmf, vector_features, 50)

In [ ]:
for topic in range(W.shape[1]):
    #max_t = np.argmax(W[:,topic])
    print "TOPIC: #",topic
    #print features[max_t][0:150]
    
    for t in range(10):
        n = int(-1-t)
        max_2t = W[:,topic].argsort()[n]
        print list(pages['Name'])[max_2t]

### LDA

In [ ]:
lda = LatentDirichletAllocation(n_topics=20, random_state=1)
W = lda.fit_transform(x_features)
H = lda.components_

In [ ]:
print_top_words(lda,vector_features, 50)

In [ ]:
for topic in range(W.shape[1]):
    #max_t = np.argmax(W[:,topic])
    print "TOPIC: #",topic
    #print features[max_t][0:150]
    
    for t in range(10):
        n = int(-1-t)
        max_2t = W[:,topic].argsort()[n]
        print list(pages['Name'])[max_2t]

In [ ]:
pages[pages['Name'] == 'Buffalo/East Side']

In [ ]:
print len(pages['text'].ix[801])

In [27]:
def f(x):
    x = re.sub(r'WikiPedia:([\s\S]*)','', x)
    x = re.sub(r'Dmoz:([\s\S]*)','', x)
    x = re.sub(r'\[([\s\S]*)\]','', x)
    x = re.sub(r'[0-9]','', x)
    x = re.sub(r'\=\=Get in\=\=([\s\S]*)(?=\=\=Get around\=\=)','',x)
    x = re.sub(r'\=\=Get around\=\=([\s\S]*)(?=\=\=See\=\=)','',x)
    x = re.sub(r'\=\=([\S ]*)\=\=', '', x)
    return x

In [28]:
pages['regex_text'] = pages['text'].apply(f)

In [29]:
pages['len'] = map(lambda x: len(x), pages['regex_text'])

In [30]:
pages2 = pages[pages['len']> 1000]

In [ ]:
len(pages2)

In [32]:
features = pages2['regex_text'].apply(lambda x: x.lower())

In [ ]:
TFIDFvect = TfidfVectorizer(stop_words=StopWords)
x_features = TFIDFvect.fit_transform(features)

In [ ]:
nmf = NMF(n_components=10, random_state=1, alpha=0, l1_ratio=0)
W = nmf.fit_transform(x_features)
H = nmf.components_

In [ ]:
vector_features = TFIDFvect.get_feature_names()

print_top_words(nmf, vector_features, 50)

In [ ]:
for topic in range(W.shape[1]):
    #max_t = np.argmax(W[:,topic])
    print "TOPIC: #",topic
    #print features[max_t][0:150]
    
    for t in range(10):
        n = int(-1-t)
        max_2t = W[:,topic].argsort()[n]
        print list(pages2['Name'])[max_2t]

In [31]:
vect = CountVectorizer(stop_words=StopWords,ngram_range=(1, 5))

In [33]:
x_features = vect.fit_transform(features)

In [34]:
nmf = NMF(n_components=10, random_state=1, alpha=0, l1_ratio=0)
W = nmf.fit_transform(x_features)
H = nmf.components_

In [37]:
vector_features = vect.get_feature_names()

print_top_words(nmf, vector_features, 50)

Topic #0:
neighborhoods restaurants especially expensive including listings although detailed largest chinese popular yorkers around famous places subway public people stores within nearby though every pages major often large times music night small known found many also find well food bars good take like one art get see nyc bus go th
Topic #1:
mdash preserves set mdash commemorates revolution mdash mdash preserves president mdash mdash contains preserves set mdash former commemorates spectacular revolution usfs mdash th century preserves including president blm mdash mdash set contains numerous situated century several largest british ancient former famous system native dating mdash civil sites first major noted along famed built years usfs well many ride set blm one th st
Topic #2:
reservations restaurants attractions restaurant available boardwalk property children fastpass resorts service guests dining though around hotels people mdash theme rides night check align also time even f

In [38]:
for topic in range(W.shape[1]):
    #max_t = np.argmax(W[:,topic])
    print "TOPIC: #",topic
    #print features[max_t][0:150]
    
    for t in range(10):
        n = int(-1-t)
        max_2t = W[:,topic].argsort()[n]
        print list(pages2['Name'])[max_2t]

TOPIC: # 0
New York City
Manhattan
San Francisco
Metro New York
Philadelphia
Seattle
Pittsburgh
Los Angeles
Atlanta
Brooklyn
TOPIC: # 1
United States National Parks
Disneyland
Finger Lakes
Montana
Early United States history
Rochester and Suburbs
San Francisco/Golden Gate
Rochester (New York)
Atlanta
Orleans County (New York)
TOPIC: # 2
Walt Disney World
Walt Disney World/Magic Kingdom
Walt Disney World/Epcot
Disneyland
Walt Disney World/Disney Springs
Universal Orlando
Walt Disney World/Animal Kingdom
Walt Disney World/Hollywood Studios
O'Hare International Airport
Las Vegas
TOPIC: # 3
Buffalo/East Side
Buffalo/South Buffalo
Buffalo/North Buffalo
Buffalo/West Side
Buffalo/Elmwood Village
Buffalo/Allentown and the Delaware District
Buffalo/Downtown
Brooklyn
Minneapolis/South
Chicago/Far Northwest Side
TOPIC: # 4
Big Bend National Park
Bandelier National Monument
Black Canyon of the Gunnison National Park
Four Corners
Isle Royale National Park
North Central New Mexico
Clarkston (Michiga

## TODO

In [ ]:
## Consider breaking text into sections (e.g. DO, EAT, etc.)

In [ ]:
## Remove get in/ get around sections

In [ ]:
## Try POS tagging

In [ ]:
## Consider NER for names of restaurants, hotels, etc. so 

In [ ]:
## Consider removing airports
## Combine city sections into one
## Remove National parks and history pages

In [ ]:
## Add European Data